Author: Ali Parandeh Presenting to: Credit Kudos

# 1. Load Data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import logging
import statsmodels as sm

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
# Construct dict of column names that need to be converted to category columns using dict comprehension
dtypes= {col:dtype for col, dtype in zip(['', '', ''], ['category'] * 3)}

In [ ]:
# Read data in, Use UCRN as index col and sort by index
df = pd.read_csv('../data/data.csv', parse_dates=[2, 4, 8], infer_datetime_format=True, dtype=dtypes)

# 2. Clean Data

## Investigating transactions Data

In [ ]:
# Basic info about each dataset
sales.info()

In [ ]:
def visualise_outliers(df: pd.DataFrame, figsize: Tuple[int, int]) -> None:
    plt.figure(figsize=figsize)
    for i, col in enumerate(df.columns):
        plt.subplot(len(df.columns), 1, i+1)
        sns.boxplot(x=df[col])
    plt.tight_layout()

In [ ]:
visualise_outliers(sales, (20, 30))

In [ ]:
def drop_outliers(df: pd.DataFrame) -> pd.DataFrame:
    # Let's drop all the outliers (say anything above 3x standard deviations above mean)
    for col in df.columns:
        if df[col].dtype == 'int' or df[col].dtype == 'float':
            mean, std = df[col].mean(), df[col].std()
            cut_off  = mean + 3 * std
            df = df[df[col] <= cut_off]
    return df

## Investigating Schemes Data

### Feature Engineering using datetime columns

Since datettime data don't work well with classification models, we need to feature engineer them to become categorical columns. One way to do this is to extract year, month and day of month or week from scheme start date. Schem start date can can be a working day or weekday or bank holiday. 

In [ ]:
from dateutil.relativedelta import relativedelta

def yearsago(years, from_date=None):
    if from_date is None:
        from_date = datetime.now()
    return (from_date - relativedelta(years=years)).year

In [ ]:
schemes['age'] = schemes.date_of_birth.apply(lambda x: yearsago(x.year))

In [ ]:
schemes['start_date_year'] = df.start_date.dt.year
schemes['start_date_month'] = df.start_date.dt.month
schemes['start_date_week'] = df.start_date.dt.isocalendar().week
schemes['start_date_day'] = schemes.start_date.dt.day
schemes['start_date_day_of_week'] = schemes.start_date.dt.day_of_week

In [ ]:
schemes['first_scheme_start_year'] = schemes.first_scheme_start.dt.year
schemes['first_scheme_start_month'] = schemes.first_scheme_start.dt.month
schemes['first_scheme_start_week'] = schemes.first_scheme_start.dt.isocalendar().week
schemes['first_scheme_start_day'] = schemes.first_scheme_start.dt.day
schemes['first_scheme_start_day_of_week'] = schemes.first_scheme_start.dt.day_of_week

## Merge Datasets

Both tables must be joined before analysis can be carried out on the features vs label (scheme status). 

In [ ]:
# Join both tables - Where there are not sales records, assume 0 sales.
schemes_sales = schemes.join(sales).fillna({col: 0 for col in sales.columns})

# 3. Explatory Data Analysis

In [ ]:
df = schemes_sales.copy()

In [ ]:
# Slice columns into numerical, datetime,  categorical
numerical_cols = list(df.select_dtypes('int').columns) + list(df.select_dtypes('float').columns)
categorical_cols = df.select_dtypes('category').columns
datetime_cols = df.select_dtypes('datetime').columns

In [ ]:
#  Visualise numerical features vs income
def visualise_numerical_correlations(df, no_cols, figsize):
    numerical_cols = list(df.select_dtypes('int').columns) + list(df.select_dtypes('float').columns)
    plt.figure(figsize=figsize)
    for i, col in enumerate(numerical_cols):
        try:
            plt.subplot(int(np.ceil(len(numerical_cols) / 3)), 3, i+1)
            plt.title(col)
            # Use violin plots to understand data distribution better.
            sns.violinplot(x="", y=col, data=df, points='inner')
        except Exception as e:
            logging.error(e)
        
    plt.tight_layout()

In [ ]:
visualise_numerical_correlations(df, 4, (20, 60))

In [ ]:
def visualise_categorical_correlations(df: pd.DataFrame, figsize: Tuple[int, int]) -> None:
    # Construct list of categorical and boolean colunmns alongside the outcome label column 
    categorical_cols = ['', '']
    
    #  One-hot encode the categorical columns to enable pairwise correlations for the first 500 rows of data      
    dummies = pd.get_dummies(df[categorical_cols])
    corr = dummies.corr()

    # Top half of the heatmap is redundant - Can use a mask to hide it         
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True

    with sns.axes_style("white"):
        f, ax = plt.subplots(figsize=figsize)
        sns.heatmap(dummies.corr(),linewidths=.1,cmap="YlGnBu", annot=True, mask=mask, vmax=.3, square=True)
    plt.yticks(rotation=0);

In [ ]:
visualise_categorical_correlations(df, (30, 20))

# 4. Build ML Model

In [ ]:
label = ''

In [ ]:
# Let's see how balanced the label classes are in the dataset
df[label].value_counts().plot(kind='barh')

Re-sampling techniques are divided in two categories:

- Under-sampling the majority class(es).
- Over-sampling the minority class.
- Combining over- and under-sampling.
- Create ensemble balanced sets.

A third-party package to help with this is [imbalanced-learn](https://github.com/scikit-learn-contrib/imbalanced-learn)

For this task, we will not investigate this as data is fake (not fully representative of the actual real-world subscriptions data)


In [ ]:
from sklearn.model_selection import train_test_split

# Drop all rows with na data
model_df = df.dropna().copy()

label = 'status_end_of_q1'

# One hot encode all categorical columns.
# Drop all columns that have no correlations with the label
X, y = pd.get_dummies(model_df.drop(columns=[label])).values, model_df[label].values

# Split data 70%-30% into training set and test set,
# As labels column (active and cancelled subscription) classes are unbalanced - we will use a stratified split to represent this unbalance in the test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0, stratify=y, shuffle=True)

In [ ]:
print ('Training cases: %d\nTest cases: %d' % (X_train.shape[0], X_test.shape[0]))

In [ ]:
# Train the model
from sklearn.linear_model import LogisticRegression

# Set regularization rate
reg = 0.01

# train a logistic regression model on the training set
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

print (model)

In [ ]:
predictions = model.predict(X_test)
print('Predicted labels: ', predictions)
print('Actual labels:    ' ,y_test)

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy: ', accuracy_score(y_test, predictions))

In [ ]:
from sklearn. metrics import classification_report

print(classification_report(y_test, predictions))

The classification report includes the following metrics for each class (False and True) on the outcome column (label)

- **Precision:** Of the predictions the model made for this class, what proportion were correct?
- **Recall:** Out of all of the instances of this class in the test dataset, how many did the model identify?
- **F1-Score:** An average metric that takes both precision and recall into account.



In [ ]:
from sklearn.metrics import confusion_matrix

# Print the confusion matrix
cm = confusion_matrix(y_test, predictions)
print (cm)

In [ ]:
from collections import Counter

In [ ]:
Counter(predictions)

In [ ]:
from sklearn.metrics import precision_score, recall_score

print("Overall Precision:",precision_score(y_test, predictions, pos_label="Cancelled"))
print("Overall Recall:",recall_score(y_test, predictions, pos_label="Cancelled"))

In [ ]:
y_scores = model.predict_proba(X_test)
print(y_scores)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1], pos_label="")

# plot ROC curve
fig = plt.figure(figsize=(6, 6))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

The ROC chart shows the curve of the true and false positive rates for different threshold values between 0 and 1. A perfect classifier would have a curve that goes straight up the left side and straight across the top. The diagonal line across the chart represents the probability of predicting correctly with a 50/50 random prediction; so we obviously want the curve to be higher than that (or our model is no better than simply guessing!).

The area under the curve (AUC) is a value between 0 and 1 that quantifies the overall performance of the model. The closer to 1 this value is, the better the model. scikit-Learn includes a function to calculate this metric.


In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

There are several ways we can improve this AUC and precision and recall scores:

1. Scaling numeric features so they're on the same scale. This prevents features with large values from producing coefficients that disproportionately affect the predictions.
2. Trying a different algorithm. Previously we used a logistic regression algorithm, which is a linear algorithm. There are many kinds of classification algorithm we could try, including:
   - **Support Vector Machine algorithms:** Algorithms that define a hyperplane that separates classes.
   - **Tree-based algorithms:** Algorithms that build a decision tree to reach a prediction
   - **Ensemble algorithms:** Algorithms that combine the outputs of multiple base algorithms to improve generalizability.
3. Hyperparameter Tuning on algorithms to build a model that fit the data better while avoiding underfitting and overfitting.
4. Instead of dropping missing data, impute missing values with statistically sound values such that we have more data to train with.
5. Feature engineering new features that correlate with the labels we want to predict on better.
6. Principal Component Analysis (PCA) to reduce features / dimensions in the data that do not help in constructing an accurate model.



# Tring a different algorithm

In [ ]:
# Train the model
from sklearn.ensemble import AdaBoostClassifier
# train a AdaBoostClassifier model on the training set
model = AdaBoostClassifier(n_estimators=5).fit(X_train, y_train)

print (model)

In [ ]:
predictions = model.predict(X_test)
y_scores = model.predict_proba(X_test)
cm = confusion_matrix(y_test, predictions)
print ('Confusion Matrix:\n',cm, '\n')
print('Accuracy:', accuracy_score(y_test, predictions))
print("Overall Precision:",precision_score(y_test, predictions, pos_label=""))
print("Overall Recall:",recall_score(y_test, predictions, pos_label=""))
auc = roc_auc_score(y_test,y_scores[:,1])
print('\nAUC: ' + str(auc))

# calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1], pos_label='')

# plot ROC curve
fig = plt.figure(figsize=(6, 6))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

## Saving and Loading model for inference


In [ ]:
import joblib

# Save the model as a pickle file
filename = '../models/income_model.pkl'
joblib.dump(model, filename)

In [ ]:
X_new = np.array([[0,0,0,0])

In [ ]:
print ('New sample: {}'.format(list(X_new[0])))

# Get a prediction
pred = model.predict(X_new)

# The model returns an array of predictions - one for each set of features submitted
# In our case, we only submitted one patient, so our prediction is the first one in the resulting array.
print('Predicted outcome is {}'.format(pred[0]))

In [ ]:
print ('New sample: {}'.format(list(X_new[1])))
pred = model.predict(X_new)
print('Predicted outcome is {}'.format(pred[1]))

# 5. Conclusion

As shown above, given an array of values based on the defined columns, one can get a prediction from the model. The model has been trained on the patterns seen in the data. In particular, the relationships between each feature column and the label column. The model is in fact a function that has mapped a hyperplane in a high dimensional search space. 

[scikit-learn](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html) has a great visual on how each algorithm can build a different model that create a different separation. In this notebook, we used logistic regression and AdaBoostClassifier algorithms.

<img src='https://scikit-learn.org/stable/_images/sphx_glr_plot_classifier_comparison_001.png' />